In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pickle as pkl
from pylatex import Document, LongTable, Package
from io import StringIO
import copy
# from reportlab.lib.pagesizes import letter
# from reportlab.pdfgen import canvas
# mode = "3_linear_probing"
# mode = "4_target_finetune"
# mode = "5_target_finetune_decay"
mode = "6_uda"
pkl_path = f'figures/{mode}.pkl'

with open(pkl_path, 'rb') as f:
    full_df = pkl.load(f)
full_df["dataset.target_envs"] = full_df["dataset.target_envs"].apply(lambda x: x[0])
interested_cols = ['dataset.name', 'dataset.target_envs', 'hue', 'fsl'] + ['training.seed'] + ["target_metric"]
full_df = full_df[interested_cols]

full_df_avg = full_df.groupby(['dataset.name', 'dataset.target_envs', 'hue', 'fsl']).mean().reset_index()
std_err = full_df.groupby(['dataset.name', 'dataset.target_envs', 'hue', 'fsl'])['target_metric'].sem().reset_index()
# Set display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

mean_pivot_df = full_df_avg.pivot(index=['dataset.name', 'dataset.target_envs', 'hue'], columns='fsl', values='target_metric')
stderr_pivot_df = std_err.pivot(index=['dataset.name', 'dataset.target_envs', 'hue'], columns='fsl', values='target_metric')

mean_pivot_df['mean'] = mean_pivot_df.mean(axis=1)
stderr_pivot_df['mean'] = stderr_pivot_df.mean(axis=1)

fsl_values = full_df_avg['fsl'].unique()


merged_df = pd.concat([mean_pivot_df, stderr_pivot_df], axis=1, keys=['mean', 'std'])


formatted_df = pd.DataFrame(index=merged_df.index)
for column in mean_pivot_df.columns:
    # formatted_df[column] = merged_df['mean'][column].astype(str) + " ± " + merged_df['std'][column].astype(str)
    # only 4 decimal places
    formatted_df[column] = merged_df['mean'][column].apply(lambda x: "{:.4f}".format(x)) + " ± " + merged_df['std'][column].apply(lambda x: "{:.4f}".format(x))

df = copy.deepcopy(formatted_df).reset_index()
fsl_list = formatted_df.columns
# group by dataset.name, dataset.target_envs, fsl, mark the max
# group_df = df.groupby(['dataset.name', 'dataset.target_envs', 'hue'])
max_value_list = []
# for name in df["dataset.name"]:
#     for e in df["dataset.target_envs"]:
#         for fsl in fsl_list:
#             group_df = df[(df["dataset.name"] == name) & (df["dataset.target_envs"] == e)][fsl]
#             mean = group_df.apply(lambda x: float(x.split(" ± ")[0]))
#             max_idx = mean.idxmax()
#             max_value = group_df.loc[max_idx]
#             max_value_list.append(max_value)
from joblib import Parallel, delayed

def max_value(df, name, e, fsl):
    group_df = df[(df["dataset.name"] == name) & (df["dataset.target_envs"] == e)][fsl]
    mean = group_df.apply(lambda x: float(x.split(" ± ")[0]))
    max_idx = mean.idxmax()
    max_value = group_df.loc[max_idx]
    return max_value

max_value_list = Parallel(n_jobs=8, backend="threading")(delayed(max_value)(df, name, e, fsl) for name in df["dataset.name"].unique() for e in df["dataset.target_envs"].unique() for fsl in fsl_list)


# max_value_list = df.groupby(['dataset.name', 'dataset.target_envs', 'hue']).apply(max_value).tolist()


# bold max_value_list
formatted_df = formatted_df.applymap(lambda x: "\\textbf{" + x + "}" if x in max_value_list else x)
for dataset in formatted_df.index.levels[0]:
    sub_df = formatted_df.loc[dataset]
    sub_df = sub_df.reset_index()
    sub_df = sub_df.set_index(['dataset.target_envs', 'hue'])
    # add bold for each dataset.target_envs, fsl combination
    env_list = sub_df.index.levels[0]
    fsl_list = sub_df.columns
    # rename fsl with 2 decimal places
    
    fsl_list = [float(fsl) if fsl != "mean" else fsl for fsl in fsl_list]
    fsl_list = ["{:.2f}".format(fsl) if fsl != "mean" else fsl for fsl in fsl_list]
    sub_df.columns = fsl_list
    sub_df.index.names = ['$E_{t}$', 'Method']
    text = sub_df.to_latex(escape=False, index=True)
    # repalce PN_ with PN-
    text = text.replace("PN_", "PN-")
    # replace NUC_ with NUC-
    text = text.replace("NUC_", "NUC-")
    with open(f'figures/{mode}_{dataset}.tex', 'w') as f:
        f.write(text)

    # sub_df.to_latex(f'figures/{mode}_{dataset}.tex', escape=False, index=True)
df.to_latex(f'figures/{mode}.tex', escape=True, index=True)

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pickle as pkl
# from pylatex import Document, LongTable, Package
from io import StringIO
import copy
# from reportlab.lib.pagesizes import letter
# from reportlab.pdfgen import canvas
# mode = "3_linear_probing"
# mode = "4_target_finetune"
# mode = "5_target_finetune_decay"
mode = "6_uda"
pkl_path = f'figures/{mode}.pkl'

with open(pkl_path, 'rb') as f:
    full_df = pkl.load(f)
full_df["dataset.target_envs"] = full_df["dataset.target_envs"].apply(lambda x: x[0])
interested_cols = ['dataset.name', 'dataset.target_envs', 'hue', 'fsl'] + ['training.seed'] + ["target_metric"]
full_df = full_df[interested_cols]

full_df_avg = full_df.groupby(['dataset.name', 'dataset.target_envs', 'hue']).mean().reset_index()
std_err = full_df.groupby(['dataset.name', 'dataset.target_envs', 'hue'])['target_metric'].sem().reset_index()
# Set display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# mean_pivot_df = full_df_avg.pivot(index=['dataset.name', 'dataset.target_envs', 'hue'], columns='fsl', values='target_metric')
# stderr_pivot_df = std_err.pivot(index=['dataset.name', 'dataset.target_envs', 'hue'], columns='fsl', values='target_metric')
mean_pivot_df = full_df_avg.pivot(index=['hue'], columns=['dataset.name', 'dataset.target_envs'], values='target_metric')
stderr_pivot_df = std_err.pivot(index=['hue'], columns=['dataset.name', 'dataset.target_envs'], values='target_metric')

# make the target_envs as the second level
# mean_pivot_df = mean_pivot_df.swaplevel(0, 1, axis=1)
# stderr_pivot_df = stderr_pivot_df.swaplevel(0, 1, axis=1)
# # drop fsl

# mean_pivot_df['mean'] = mean_pivot_df.mean(axis=1)
# stderr_pivot_df['mean'] = stderr_pivot_df.mean(axis=1)

# fsl_values = full_df_avg['fsl'].unique()


# merged_df = pd.concat([mean_pivot_df, stderr_pivot_df], axis=1, keys=['mean', 'std'])


# formatted_df = pd.DataFrame(index=merged_df.index)
# for column in mean_pivot_df.columns:
#     # formatted_df[column] = merged_df['mean'][column].astype(str) + " ± " + merged_df['std'][column].astype(str)
#     # only 4 decimal places
#     formatted_df[column] = merged_df['mean'][column].apply(lambda x: "{:.4f}".format(x)) + " ± " + merged_df['std'][column].apply(lambda x: "{:.4f}".format(x))

# df = copy.deepcopy(formatted_df).reset_index()
# fsl_list = formatted_df.columns
# # group by dataset.name, dataset.target_envs, fsl, mark the max
# # group_df = df.groupby(['dataset.name', 'dataset.target_envs', 'hue'])
# max_value_list = []
# # for name in df["dataset.name"]:
# #     for e in df["dataset.target_envs"]:
# #         for fsl in fsl_list:
# #             group_df = df[(df["dataset.name"] == name) & (df["dataset.target_envs"] == e)][fsl]
# #             mean = group_df.apply(lambda x: float(x.split(" ± ")[0]))
# #             max_idx = mean.idxmax()
# #             max_value = group_df.loc[max_idx]
# #             max_value_list.append(max_value)
# from joblib import Parallel, delayed

# def max_value(df, name, e, fsl):
#     group_df = df[(df["dataset.name"] == name) & (df["dataset.target_envs"] == e)][fsl]
#     mean = group_df.apply(lambda x: float(x.split(" ± ")[0]))
#     max_idx = mean.idxmax()
#     max_value = group_df.loc[max_idx]
#     return max_value

# print(df)
# max_value_list = Parallel(n_jobs=8, backend="threading")(delayed(max_value)(df, name, e, fsl) for name in df["dataset.name"].unique() for e in df["dataset.target_envs"].unique() for fsl in fsl_list)



# # bold max_value_list
# formatted_df = formatted_df.applymap(lambda x: "\\textbf{" + x + "}" if x in max_value_list else x)
# # for dataset in formatted_df.index.levels[0]:
# sub_df = formatted_df
# sub_df = sub_df.reset_index()
# sub_df = sub_df.set_index(['dataset.name','dataset.target_envs', 'hue'])
# # add bold for each dataset.target_envs, fsl combination
# env_list = sub_df.index.levels[0]
# fsl_list = sub_df.columns
# # rename fsl with 2 decimal places

# fsl_list = [float(fsl) if fsl != "mean" else fsl for fsl in fsl_list]
# fsl_list = ["{:.2f}".format(fsl) if fsl != "mean" else fsl for fsl in fsl_list]
# sub_df.columns = fsl_list
# sub_df.index.names = ['D', '$E_{t}$', 'Method']
# text = sub_df.to_latex(escape=False, index=True)
# # repalce PN_ with PN-
# text = text.replace("PN_", "PN-")
# # replace NUC_ with NUC-
# text = text.replace("NUC_", "NUC-")
# with open(f'figures/{mode}.tex', 'w') as f:
#     f.write(text)
#     # sub_df.to_latex(f'figures/{mode}_{dataset}.tex', escape=False, index=True)
# df.to_latex(f'figures/{mode}.tex', escape=True, index=True)

In [5]:
mean_pivot_df
# drop mean in mean_pivot_df
# mean_pivot_df = mean_pivot_df.drop(columns=["mean"])
# calculate the mean for each target_envs in each dataset.name
df = mean_pivot_df
mean_df = df.groupby(level=0, axis=1).mean()
mean_df.columns = pd.MultiIndex.from_product([mean_df.columns, ['Mean']], names=['First', 'Second'])
df = pd.merge(df, mean_df, left_index=True, right_index=True)
# rename second level as str
df.columns = df.columns.set_levels(df.columns.levels[1].astype(str), level=1)
# sort the columns
df = df.sort_index(axis=1)
# mark the max in each Mean column
max_value_list = []
for name in df.columns.levels[0]:
    group_df = df[name]
    print(name)
    mean_col = group_df["Mean"]
    max_idx = mean_col.idxmax()
    max_value = df[name].loc[max_idx]["Mean"]
    max_value_list.append("{:.4f}".format(max_value))

# 4 decimal places and convert to str
df = df.applymap(lambda x: "{:.4f}".format(x))
df = df.applymap(lambda x: str(x) if x not in max_value_list else x)
df = df.applymap(lambda x: "\\textbf{" + str(x) + "}" if x in max_value_list else x)

df.index.names = ['Method']
# df.columns.names =
# set first level 

text = df.to_latex(escape=False, index=True, multirow=True)

# repalce PN_ with PN-
text = text.replace("PN_", "PN-")
# replace NUC_ with NUC-
text = text.replace("NUC_", "NUC-")
with open(f'figures/{mode}.tex', 'w') as f:
    f.write(text)

officehome
pacs
terraincognita
vlcs


In [6]:
df

officehome                                             pacs          \
                  0       1       2       3             Mean       0       1   
Method                                                                         
DANN         0.6091  0.0297  0.6599  0.7408           0.5099  0.1463  0.1026   
DIWA         0.6914  0.5767  0.7838  0.7982           0.7125  0.9024  0.8162   
ERM          0.6914  0.5240  0.7793  0.8211           0.7039  0.8244  0.7906   
NUC_0.01     0.6996  0.5446  0.8018  0.8119           0.7145  0.8244  0.7521   
NUC_0.1      0.7284  0.5423  0.7995  0.8188           0.7223  0.8195  0.8120   
PN_y         0.7243  0.4920  0.8063  0.8417           0.7161  0.7707  0.8120   
PN_y-s       0.7284  0.5057  0.8108  0.8486  \textbf{0.7234}  0.7659  0.8120   
PN_y-s-e     0.7160  0.5034  0.8086  0.8463           0.7186  0.7756  0.8120   

                                          terraincognita                  \
               2       3             Mean              0       1       2   
Method                                                                     
DANN      0.6707  0.2112           0.2827         0.0886  0.0770  0.2040   
DIWA      0.9880  0.8244  \textbf{0.8828}         0.5781  0.4517  0.5995   
ERM       0.9641  0.8270           0.8515         0.5190  0.4415  0.5693   
NUC_0.01  0.9820  0.7888           0.8368         0.2025  0.4138  0.5516   
NUC_0.1   0.9880  0.7990           0.8546         0.6181  0.4969  0.6121   
PN_y      0.9641  0.7150           0.8154         0.5570  0.5277  0.5340   
PN_y-s    0.9641  0.6997           0.8104         0.5612  0.5277  0.5290   
PN_y-s-e  0.9641  0.6921           0.8109         0.5675  0.5246  0.5365   

                                     vlcs                          \
               3             Mean       0       1       2       3   
Method                                                              
DANN      0.2806           0.1626  0.9437  0.3636  0.4006  0.4438   
DIWA      0.4354           0.5162  0.9718  0.6014  0.7350  0.8047   
ERM       0.3861           0.4789  0.9718  0.6503  0.7098  0.8136   
NUC_0.01  0.2177           0.3464  0.3169  0.6692  0.7652  0.1391   
NUC_0.1   0.2177           0.4862  0.9789  0.6842  0.7287  0.8077   
PN_y      0.4762           0.5237  0.9789  0.5594  0.7382  0.8166   
PN_y-s    0.4728           0.5227  0.9859  0.5594  0.7382  0.8343   
PN_y-s-e  0.4779  \textbf{0.5266}  0.9789  0.5594  0.7382  0.8373   

                           
                     Mean  
Method                     
DANN               0.5379  
DIWA               0.7782  
ERM                0.7864  
NUC_0.01           0.4726  
NUC_0.1   \textbf{0.7999}  
PN_y               0.7733  
PN_y-s             0.7795  
PN_y-s-e           0.7784